In [2]:
from pyspark import SparkContext

In [3]:
import numpy as np
import pandas as pd

In [4]:
sc = SparkContext('local')

22/07/26 13:35:06 WARN Utils: Your hostname, master resolves to a loopback address: 127.0.1.1; using 192.168.139.128 instead (on interface ens33)
22/07/26 13:35:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/26 13:35:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
dt = [(1,2),(3,4),(5,6),(1,7),(3,4)]
nRdd = sc.parallelize(dt)
nRdd.collect()

[(1, 2), (3, 4), (5, 6), (1, 7), (3, 4)]

In [6]:
# 튜플 앞에값을 key로 생각하고, 같읕 key끼리 값을 누적합
newRdd = nRdd.reduceByKey(lambda a,b:a+b) #transform
newRdd.collect()

[(1, 9), (3, 8), (5, 6)]

In [7]:
newRdd1 = nRdd.flatMap(lambda v:v).map(lambda v: (v,1))
newRdd1.collect()

[(1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (1, 1),
 (7, 1),
 (3, 1),
 (4, 1)]

In [8]:
newRdd1.reduceByKey(lambda a,b:a+b).collect()

[(1, 2), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1)]

In [9]:
# 딕셔너리로　출력
d = newRdd1.countByKey() #action함수
d

defaultdict(int, {1: 2, 2: 1, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1})

In [10]:
d[1]

2

In [11]:
# 키순서대로 정렬（오름차순）
newRdd1.sortByKey().collect() #transform

[(1, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (3, 1),
 (4, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1)]

In [12]:
# 키순서대로 정렬（내림차순）
newRdd1.sortByKey(ascending=False).collect() #transform

[(7, 1),
 (6, 1),
 (5, 1),
 (4, 1),
 (4, 1),
 (3, 1),
 (3, 1),
 (2, 1),
 (1, 1),
 (1, 1)]

In [13]:
nRdd.collect()

[(1, 2), (3, 4), (5, 6), (1, 7), (3, 4)]

In [14]:
# 값순서대로 정렬
nRdd.sortBy(lambda v:v[1]).collect() #transform

[(1, 2), (3, 4), (3, 4), (5, 6), (1, 7)]

### word count

In [15]:
txtRdd = sc.textFile('./data/aa.txt')
txtRdd.collect()

['나는 자랑스런', '태극기 앞에', '조국과 태극기', '몸과 나는']

In [16]:
txtRdd.map(lambda v: v.split()).collect()

[['나는', '자랑스런'], ['태극기', '앞에'], ['조국과', '태극기'], ['몸과', '나는']]

In [17]:
txtRdd.flatMap(lambda v:v.split()).collect()

['나는', '자랑스런', '태극기', '앞에', '조국과', '태극기', '몸과', '나는']

In [18]:
txtRdd.flatMap(lambda v:v.split()).map(lambda v:(v,1)).collect()

[('나는', 1),
 ('자랑스런', 1),
 ('태극기', 1),
 ('앞에', 1),
 ('조국과', 1),
 ('태극기', 1),
 ('몸과', 1),
 ('나는', 1)]

In [19]:
txtRdd.flatMap(lambda v:v.split()).map(lambda v:(v,1)).countByKey()

defaultdict(int, {'나는': 2, '자랑스런': 1, '태극기': 2, '앞에': 1, '조국과': 1, '몸과': 1})

In [20]:
txtRdd = sc.textFile('./data/k.txt')

In [21]:
txtRdd.flatMap(lambda v:v.split()).map(lambda v:(v,1)).reduceByKey(lambda a,b: a+b).sortBy(lambda a:a[1], ascending=False).take(10)

[('원내대표는', 7),
 ('질문에', 7),
 ('것은', 5),
 ('여야', 4),
 ('하는', 4),
 ('말했다.', 4),
 ('3당', 3),
 ('위해', 3),
 ('문', 3),
 ('이인영', 3)]

In [22]:
txtRdd.flatMap(lambda v:v.split()).map(lambda v:v.replace('"','')).map(lambda v:v.replace("'","")).map(lambda v:v.replace(".","")).map(lambda v:(v,1)).reduceByKey(lambda a,b: a+b).sortBy(lambda a:a[1], ascending=False).take(10)

[('원내대표는', 7),
 ('질문에', 7),
 ('것은', 5),
 ('여야', 4),
 ('하는', 4),
 ('말했다', 4),
 ('3당', 3),
 ('위해', 3),
 ('문', 3),
 ('이인영', 3)]

In [39]:
sc.stop()